In [1]:
!which python

/Users/priya/Downloads/UC Davis/Assignments/venv/bin/python


In [6]:
pip install -r requirements.txt

You should consider upgrading via the '/Users/priya/Downloads/UC Davis/Assignments/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Input the dish item from the user

Open the search result from Doordash website

In [ ]:
import undetected_chromedriver as uc
import chromedriver_py  # this imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.6943.142 Safari/537.36")

# Get ChromeDriver path from chromedriver_py
driver_path = chromedriver_py.binary_path

# Set up undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(driver_executable_path=driver_path, options=options)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page
time.sleep(random.uniform(2, 4))

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Give time for the page to load
time.sleep(random.uniform(2, 4))

# Close browser
input("Press Enter to close the browser...")
driver.quit()


This page has a lot of restaurants that serves sushi

- Fetch the restaurant id from the element

In [10]:
import undetected_chromedriver as uc  # Correct import statement
import chromedriver_py  # This imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Set up Chrome options
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Get the ChromeDriver path using chromedriver_py
driver_path = chromedriver_py.binary_path

# Initialize undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(executable_path=driver_path, options=options, timeout=120)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page to mimic human-like browsing
time.sleep(random.uniform(2, 4))  # Wait between 3 to 7 seconds randomly

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Give time for the page to load and handle any security checks that might still appear
time.sleep(random.uniform(2, 4))  # Wait again to mimic a real user

# Fetch all restaurant ID links
try:
    # Use XPath to find all <a> tags containing the store IDs
    restaurant_elements = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/store/')]")
    
    # Extract the restaurant IDs from the href attributes
    restaurant_ids = []
    for element in restaurant_elements:
        href = element.get_attribute('href')
        # Extract the ID from the href by splitting the string
        if '/store/' in href:
            restaurant_id = href.split('/store/')[1].split('?')[0]
            restaurant_ids.append(restaurant_id)

    # Remove duplicates by converting to a set and then back to a list
    unique_restaurant_ids = list(set(restaurant_ids))

    # Print the first 5 unique restaurant IDs
    print(f"First 5 restaurant IDs for '{search_input}':")
    for restaurant_id in unique_restaurant_ids[:5]:
        print(restaurant_id)

    # Print total number of unique restaurant IDs found
    print(f"\nTotal number of unique restaurant IDs found: {len(unique_restaurant_ids)}")

except Exception as e:
    print(f"Error extracting restaurant IDs: {e}")

# Wait for the user to press Enter before closing the browser
input("\nPress Enter to close the browser...")

# Close the browser after the user presses Enter
driver.quit()


First 5 restaurant IDs for 'sushi':
193470
240183
956708
718398
869275

Total number of unique restaurant IDs found: 10


Create a JSON for every restaurant and append all the details - ID, Name, Ratings, No of Ratings, Delivery Time, Distance

In [ ]:
import undetected_chromedriver as uc  # Correct import statement
import chromedriver_py  # This imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Set up Chrome options
options = Options()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Get the ChromeDriver path using chromedriver_py
driver_path = chromedriver_py.binary_path

# Initialize undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(executable_path=driver_path, options=options, timeout=120)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page to mimic human-like browsing
time.sleep(random.uniform(2, 4))  # Wait between 3 to 7 seconds randomly

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Give time for the page to load and handle any security checks that might still appear
time.sleep(random.uniform(2, 4))  # Wait again to mimic a real user

# Initialize the list to hold restaurant data
restaurants_data = []

# Fetch all restaurant ID links
try:
    # Use XPath to find all <a> tags containing the store IDs
    restaurant_elements = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/store/')]")
    print(restaurant_elements)
    
    # Extract the restaurant IDs and additional data from the href attributes
    for element in restaurant_elements:
        href = element.get_attribute('href')
        print('href', href)
        # Extract the ID from the href by splitting the string
        if '/store/' in href:
            restaurant_id = href.split('/store/')[1].split('?')[0]
            url = href
            
            # Extract restaurant name
            try:
                name_element = element.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/main/main/div[3]/div[2]/div[1]/div/div/div[3]/div[1]/div/div/div[7]/div/div[2]/div[2]/a/div/div[1]/span")
                restaurant_name = name_element.text.strip() if name_element else ''
            except:
                restaurant_name = ''

            # Extract ratings
            try:
                rating_element = element.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/main/main/div[3]/div[2]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/div/div[2]/div[2]/a/div/div[2]/span/div/div[1]/div/div/span")
                rating = rating_element.text.strip() if rating_element else ''
            except:
                rating = ''

            # Extract number of ratings
            try:
                no_of_ratings_element = element.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/main/main/div[3]/div[2]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/div/div[2]/div[2]/a/div/div[2]/span/div/div[1]/div/span")
                no_of_ratings = no_of_ratings_element.text.strip() if no_of_ratings_element else ''
            except:
                no_of_ratings = ''
            
            # Extract distance
            try:
                distance_element = element.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/main/main/div[3]/div[2]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/div/div[2]/div[2]/a/div/div[2]/span/div/div[2]/div/span")
                distance = distance_element.text.strip() if distance_element else ''
            except:
                distance = ''
            
            # Extract delivery time
            try:
                delivery_time_element = element.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/main/main/div[3]/div[2]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/div/div[2]/div[2]/a/div/div[2]/span/div/div[3]/div/span")
                delivery_time = delivery_time_element.text.strip() if delivery_time_element else ''
            except:
                delivery_time = ''
            
            # Append the restaurant data to the list as a JSON object
            restaurant_data = {
                "restaurant_id": restaurant_id,
                "URL": url,
                "restaurant_name": restaurant_name,
                "ratings": rating,
                "no_of_ratings": no_of_ratings,
                "distance": distance,
                "delivery_time": delivery_time,
                "price": ''  
            }
            restaurants_data.append(restaurant_data)

    # Print the list of restaurant data
    print(f"First 5 restaurant data for '{search_input}':")
    for restaurant in restaurants_data[:5]:
        print(restaurant)

    # Print total number of restaurants
    print(f"\nTotal number of restaurants found: {len(restaurants_data)}")

except Exception as e:
    print(f"Error extracting restaurant data: {e}")

# Wait for the user to press Enter before closing the browser
input("\nPress Enter to close the browser...")

# Close the browser after the user presses Enter
driver.quit()


[<undetected_chromedriver.webelement.WebElement (session="bb4f982c805167566ac179489e95049f", element="f.6BE64E4A090A9C48B5FEE2C147EB5487.d.5ACD1576441517320BF8A2CEC9F84649.e.46")>, <undetected_chromedriver.webelement.WebElement (session="bb4f982c805167566ac179489e95049f", element="f.6BE64E4A090A9C48B5FEE2C147EB5487.d.5ACD1576441517320BF8A2CEC9F84649.e.47")>, <undetected_chromedriver.webelement.WebElement (session="bb4f982c805167566ac179489e95049f", element="f.6BE64E4A090A9C48B5FEE2C147EB5487.d.5ACD1576441517320BF8A2CEC9F84649.e.48")>, <undetected_chromedriver.webelement.WebElement (session="bb4f982c805167566ac179489e95049f", element="f.6BE64E4A090A9C48B5FEE2C147EB5487.d.5ACD1576441517320BF8A2CEC9F84649.e.49")>, <undetected_chromedriver.webelement.WebElement (session="bb4f982c805167566ac179489e95049f", element="f.6BE64E4A090A9C48B5FEE2C147EB5487.d.5ACD1576441517320BF8A2CEC9F84649.e.50")>, <undetected_chromedriver.webelement.WebElement (session="bb4f982c805167566ac179489e95049f", element

KeyboardInterrupt: Interrupted by user

Navigate into each restaurant through the url created from restaurant ID

In [19]:
import undetected_chromedriver as uc  # Correct import statement
import time
import random
from selenium.webdriver.chrome.options import Options

# Set up Chrome options (without headless mode)
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Initialize undetected ChromeDriver
driver = uc.Chrome(options=options)

# Open the DoorDash store page for a specific restaurant ID
restaurant_id = '956708'
restaurant_url = f"https://www.doordash.com/store/{restaurant_id}"
driver.get(restaurant_url)

# Wait for the page to load (optional, you can adjust the wait time based on your needs)
time.sleep(random.uniform(3, 7))  # Wait between 3 to 7 seconds to mimic human-like browsing

# Wait for the URL to update automatically to the new format (store name + restaurant ID)
time.sleep(3)

# Extract the updated URL from the browser
updated_url = driver.current_url

# Extract the restaurant name from the URL
# The format of the URL is: https://www.doordash.com/store/<restaurant_name>/<restaurant_id>/
# Split the URL to get the restaurant name and ID
url_parts = updated_url.split('/')

# The restaurant name is the part before the restaurant ID in the URL
restaurant_name = url_parts[4]  # The name will be in the 4th part of the URL

# Print the restaurant name and ID
print(f"Restaurant Name: {restaurant_name}")
print(f"Restaurant ID: {restaurant_id}")

# Wait for the user to press Enter before closing the browser
input("\nPress Enter to close the browser...")

# Close the browser after the user presses Enter
driver.quit()


Restaurant Name: i-love-sushi-davis-956708
Restaurant ID: 956708


Search the item from the restaurant page